In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.special import ndtri
from scipy.stats import gaussian_kde, gamma, uniform
from scipy.linalg import cholesky, LinAlgError, eigvalsh, eigh, inv
from scipy.special import gamma as gamma_func
from scipy.special import yv
from statsmodels.stats.correlation_tools import cov_nearest
import time
import sys
import os
import six
import corner
import pymc3 as pm
from mpi4py import MPI

%matplotlib notebook


def f(x):
    """The function to predict."""
    return x * np.sin(x)

def rbf(theta,x):
    
    return theta[0]**2 * np.exp(-0.5 * theta[1]**-2 * np.subtract.outer(x,x)**2)

def periodic(theta,x):
    
    return theta[0]**2 * np.exp(-2 * theta[1]**-2 * np.sin(np.pi * np.subtract.outer(x,x) / theta[2])**2)

def local_periodic(theta,x):
    
    return rbf(theta,x) * periodic(theta,x)

def matern(theta, x):

    u = np.sqrt(2 * theta[0]) * abs(np.subtract.outer(x,x) / theta[1])
    u[u == 0.0] += np.finfo(float).eps
    return 2**(1-theta[0]) / gamma_func(theta[0]) * (u)**theta[0] * yv(theta[0], u)

K_XX_error = []

def GP(kernel, theta, data, mu_prior=[], sigma=[]):
    
    global K_XX_error
    
    # Define test points (XT) and training data (X, y)
    XT, X, y = data
    n = len(X)
    
    # Calculate cov. matrix for join distribution
    K = kernel(theta, np.concatenate((X, XT)))
    
    # For non-noisy training data set sigma = 0
    if len(sigma)==0:
        sigma = np.zeros(n)
    if len(mu_prior)==0:
        mu_prior = np.zeros(n)
    
    # Sub-matrices of joint distribution, using cholesky decomp. for inversion
    K_XTX = K[n:,:n]
    K_XX = K[:n,:n]+np.diag(sigma**2)
    try:
        ch_K_XX = cholesky(K_XX, lower=True)#+np.diag(np.ones(len(K_XX)))*1E-10, lower=True)
    except:
        display(theta)
        display(K_XX)
        display(eigvalsh(K_XX))
        K_XX_error = K_XX
        
    K_XX_inv = inv(ch_K_XX.T) @ inv(ch_K_XX)#inv(K[:n,:n]+np.diag(sigma**2))
    K_XXT = K[:n,n:]
    K_XTXT = K[n:,n:]
    
    # Find conditioned mean function and covariance matrix
    m = K_XTX @ K_XX_inv @ (y-mu_prior)
    K = K_XTXT - K_XTX @ K_XX_inv @ K_XXT
    
    return (m, np.sqrt(np.diag(K)))

def plotposts(samples, **kwargs):
    """
    Function to plot posteriors using corner.py and scipy's gaussian KDE function.
    """
    
    fig = corner.corner(samples, labels=[r'$s$', r'$l$', r'$p$'], hist_kwargs={'density': True}, **kwargs)

    # plot KDE smoothed version of distributions
    #for axidx, samps in zip([0, 6], samples.T):
    #    kde = gaussian_kde(samps)
    #    xvals = fig.axes[axidx].get_xlim()
    #    xvals = np.linspace(xvals[0], xvals[1], 100)
    #    fig.axes[axidx].plot(xvals, kde(xvals), color='firebrick')

In [34]:
X = np.arange(0,15,0.5)
XT = np.arange(X[0],X[-1]+1,0.01)
dy = 0.5
y = X*np.sin(X)+np.random.normal(scale=dy,size=len(X))
data = (XT, X, y)
sigma = np.ones(len(X))*dy
theta = [5/2, 1]
mu_prior = np.zeros(len(X))

gp = GP(matern, theta, data, mu_prior=mu_prior, sigma=sigma)
plt.figure()
plt.plot(X, y, 'k.')
plt.plot(XT, gp[0], c='r')
plt.fill_between(XT, gp[0]+gp[1]*2, gp[0]-gp[1]*2, color='r', alpha=0.2)
plt.plot(XT, XT*np.sin(XT), c='b')

[2.5, 1]

array([[  -0.38661977,   -0.80249943,   -1.38193587,   -1.260961  ,
           1.90687494,    6.88554872,    6.35880595,   -4.63752346,
         -17.12156726,  -12.94102319,   11.75274309,   32.4098457 ,
          18.37128071,  -25.6304845 ,  -51.82759003,  -19.40800108,
          47.86032694,   73.19556719,   12.60706381,  -78.89972666,
         -93.20587025,    5.22314833,  117.83001799,  107.68546001,
         -36.55495747, -162.24029769, -111.97805346,   82.70585665,
         208.25761673,  101.41633334],
       [  -0.80249943,   -0.38661977,   -0.80249943,   -1.38193587,
          -1.260961  ,    1.90687494,    6.88554872,    6.35880595,
          -4.63752346,  -17.12156726,  -12.94102319,   11.75274309,
          32.4098457 ,   18.37128071,  -25.6304845 ,  -51.82759003,
         -19.40800108,   47.86032694,   73.19556719,   12.60706381,
         -78.89972666,  -93.20587025,    5.22314833,  117.83001799,
         107.68546001,  -36.55495747, -162.24029769, -111.97805346,
         

array([-6.58446424e+02, -5.47151490e+02, -1.08755075e+02, -8.96104477e+01,
        2.50000000e-01,  2.50000000e-01,  2.50000000e-01,  2.50000000e-01,
        2.50000000e-01,  2.50000000e-01,  2.50000000e-01,  2.50000000e-01,
        2.50000000e-01,  2.50000000e-01,  2.50000000e-01,  2.50000000e-01,
        2.50000000e-01,  2.50000000e-01,  2.50000000e-01,  2.50000000e-01,
        2.50000000e-01,  2.50000000e-01,  2.50000000e-01,  2.50000000e-01,
        2.50000000e-01,  2.50000000e-01,  2.50000000e-01,  2.50000000e-01,
        6.35556332e+02,  7.50808512e+02])

UnboundLocalError: local variable 'ch_K_XX' referenced before assignment

In [7]:
X = np.arange(0,175,0.25)

dy= 0.5
y = f(X) + np.random.normal(scale=dy, size=len(X))
sigma = np.ones(len(y))*dy

def prior_transform(theta):

    return gamma.ppf(theta,5.5)

def loglikelihood(theta):
    
    # Define global variables
    global X,y,sigma
    data = (X[1::2],X[::2],y[::2])
    sigma_training = sigma[::2]
    y_test = y[1::2]
    sigma_test = sigma[1::2]

    # normalisation
    norm = -0.5*len(data[0])*np.log(2*np.pi) - np.sum(np.log(sigma_test))
    gp = GP(rbf, theta, data, sigma=sigma_training)
    # chi-squared
    chisq = np.sum(((y_test-gp[0])/sigma_test)**2)

    return norm - 0.5*chisq, []

In [8]:

gamma.ppf(0.01, 5.5)

1.52674205332034